In [1]:
'''
GAT Prediction Cross Validation

Code Adapted from Alva Yan's Post-Covid Sentiment Analysis,
https://github.com/AlvaYan/postCOVIDSentiAnalysis, and
https://github.com/AlvaYan/Sentiment-Analysis-GNN-During-COVID19 with permission.

Code was adapted while referencing public examples from the
Keras documentation on GitHub:
https://github.com/fchollet/keras/blob/master/examples
'''

#Cross validation. From a set of candidate parameters, see which parameter set performs the best s
#make sure model code (gat, layers, base_gattn) is in path "path_code" 
#Code in this nodebook is modified from: https://github.com/Jhy1993/HAN 

path_result='C:\\Users\\yunfa\\dsc180b\\results\\'
path_data="C:\\Users\\yunfa\\dsc180b\\data"
path_code="C:\\Users\\yunfa\\dsc180b"
path_temp="C:\\Users\\yunfa\\dsc180b\\temp"

import time
import numpy as np
import tensorflow as tf
import os
import random
os.chdir(path_code)
import gat
import imp
imp.reload(gat)
from gat import GAT, HeteGAT, HeteGAT_multi  # or * for that matter
import process
import importlib
importlib.reload(process)
import numpy as np
import pickle
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# jhy data
import scipy.io as sio
import scipy.sparse as sp


def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

def load_data_dblp(fold,path=None):
    import pandas as pd
    import pickle
    import scipy.sparse
    from scipy.sparse import csc_matrix
    from scipy import sparse
    import numpy as np 
    import os
    import csv 
    import numpy as np
    # schools = ["notredame","uofm","columbia","dartmouth","UCSD","berkeley","Harvard","ucla"]
    schools = ["notredame","uofm","columbia","UCSD","berkeley","Harvard","ucla"]
    s=schools[1]#use this index to control which network we are going to run.
    t="2019"
    
    os.chdir(path_temp)
    
    name="feature_"+s+t+".csv"
    truefeatures=pd.read_csv(name,skip_blank_lines=True,header=None).values
    N=truefeatures.shape[0]
    
    os.chdir(path_data)
    
    name="label_emt3_"+s+t+"_cm.csv"
    truelabels=pd.read_csv(name,skip_blank_lines=True,header=None)
    
    name="CCasym_"+s+t+".npz"
    dat_cc=scipy.sparse.load_npz(name);dat_cc=dat_cc.toarray();
    rownetworks = [(dat_cc)]
    
    #TODO: dartmouth change to 2019, missing 2020
    os.chdir(path_data)

    y=truelabels    
    name="train_index_"+s+t+"_cm.p"
    train_idx = pickle.load(open(name,"rb"));train_idx=np.array(train_idx)
    name="to_be_labeled_index_"+s+t+"_cm.p"
    val_idx = pickle.load(open(name,"rb"));val_idx=np.array(val_idx)
    name="test_index_"+s+t+"_cm.p"
    #name="test_index_"+s+y+"_cm.p"#use this to do 3 class CV
    test_idx = pickle.load(open(name,"rb"));test_idx=np.array(test_idx)
    import random
    random.seed(3407)
    random.shuffle(train_idx)
    indexes=[]
    temp=sample_mask(train_idx, y.shape[0])
    tt=y.iloc[temp, :]
    
    t=np.sum(tt,axis=0)
    n_neg=t[0]
    n_pos=t[1]
    train_idx=tuple(train_idx)
    train_idx_tm=list(train_idx)
    for k in range(kfolds):
    
        bgt_neg=round(n_neg/kfolds)
        bgt_pos=round(n_pos/kfolds)
        if k==kfolds-1:
            bgt_neg=n_neg-k*round(n_neg/kfolds)
            bgt_pos=n_pos-k*round(n_pos/kfolds)
        group_index=[]
        for train in train_idx:
            if list(y.iloc[train,]).index(1)==0 and bgt_neg>0:
                group_index.append(train)
                train_idx_tm.remove(train)
                bgt_neg-=1
            elif list(y.iloc[train,]).index(1)==1 and bgt_pos>0:
                group_index.append(train)
                train_idx_tm.remove(train)
                bgt_pos-=1
        train_idx=tuple(train_idx_tm)
        indexes.append(group_index)
    
    val_idx=np.array(indexes[fold])
    import itertools
    train_idx=list(itertools.chain.from_iterable(indexes))
    for t in val_idx: train_idx.remove(t)
    train_idx=np.array(train_idx)
    
    train_mask = sample_mask(train_idx, y.shape[0])
    val_mask = sample_mask(val_idx, y.shape[0])
    test_mask = sample_mask(test_idx, y.shape[0])
    
    y_train = np.zeros(y.shape)
    y_val = np.zeros(y.shape)
    y_test = np.zeros(y.shape)
    y=y.values
    y_train[train_mask, :] = y[train_mask, :]
    y_val[val_mask, :] = y[val_mask, :]
    y_test[test_mask, :] = y[test_mask, :]
    truefeatures_list = [truefeatures, truefeatures]#, truefeatures, truefeatures]# ?? why copy three times? First for center node, 2 for each metapath.    
    os.chdir(path_code)
    return rownetworks, truefeatures_list, y_train, y_val, y_val, train_mask, val_mask, val_mask

def HAN(nb_epochs= 100,l2_coef= 0.05,lr= 0.005,hid_units= [8],n_heads= [4, 1] ,ffd_drop_set=0,attn_drop_set=0,fold=0, weight=1):

    dataset = 'acm'
    featype = 'fea'
    checkpt_file = path_result+'acm_allMP_multi_fea_2cls.ckpt'
    batch_size = 1
    patience = 100
    residual = False
    nonlinearity = tf.nn.elu
    model = HeteGAT_multi

    adj_list, fea_list, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data_dblp(fold=fold)
    if featype == 'adj':
        fea_list = adj_list
    nb_nodes = fea_list[0].shape[0]
    ft_size = fea_list[0].shape[1]
    nb_classes = y_train.shape[1]

    fea_list = [fea[np.newaxis] for fea in fea_list]
    adj_list = [adj[np.newaxis] for adj in adj_list]
    y_train = y_train[np.newaxis]
    y_val = y_val[np.newaxis]
    y_test = y_test[np.newaxis]
    train_mask = train_mask[np.newaxis]
    val_mask = val_mask[np.newaxis]
    test_mask = test_mask[np.newaxis]

    biases_list = [process.adj_to_bias(adj, [nb_nodes], nhood=1) for adj in adj_list]
    with tf.Graph().as_default():
        with tf.name_scope('input'):
            ftr_in_list = [tf.placeholder(dtype=tf.float32,
                                          shape=(batch_size, nb_nodes, ft_size),
                                          name='ftr_in_{}'.format(i))
                           for i in range(len(fea_list))]
            bias_in_list = [tf.placeholder(dtype=tf.float32,
                                           shape=(batch_size, nb_nodes, nb_nodes),
                                           name='bias_in_{}'.format(i))
                            for i in range(len(biases_list))]
            lbl_in = tf.placeholder(dtype=tf.int32, shape=(
                batch_size, nb_nodes, nb_classes), name='lbl_in')
            msk_in = tf.placeholder(dtype=tf.int32, shape=(batch_size, nb_nodes),
                                    name='msk_in')
            attn_drop = tf.placeholder(dtype=tf.float32, shape=(), name='attn_drop')
            ffd_drop = tf.placeholder(dtype=tf.float32, shape=(), name='ffd_drop')
            is_train = tf.placeholder(dtype=tf.bool, shape=(), name='is_train')
        # forward
        logits, final_embedding, att_val = model.inference(ftr_in_list, nb_classes, nb_nodes, is_train,
                                                       attn_drop, ffd_drop,
                                                       bias_mat_list=bias_in_list,
                                                       hid_units=hid_units, n_heads=n_heads,
                                                       residual=residual, activation=nonlinearity)

        # cal masked_loss
        log_resh = tf.reshape(logits, [-1, nb_classes])
        lab_resh = tf.reshape(lbl_in, [-1, nb_classes])
        msk_resh = tf.reshape(msk_in, [-1])
        loss = model.masked_softmax_cross_entropy(log_resh, lab_resh, msk_resh, weight)
        predicted=tf.argmax(log_resh, 1)
        accuracy = model.masked_accuracy(log_resh, lab_resh, msk_resh)
        # optimzie
        train_op = model.training(loss, lr, l2_coef)
    
        saver = tf.train.Saver()

        init_op = tf.group(tf.global_variables_initializer(),
                       tf.local_variables_initializer())

        vlss_mn = np.inf
        vacc_mx = 0.0
        curr_step = 0

        with tf.Session(config=config) as sess:
            sess.run(init_op)

            train_loss_avg = 0
            train_acc_avg = 0
            val_loss_avg = 0
            val_acc_avg = 0
            val_pred_avg = 0
            for epoch in range(nb_epochs):
                tr_step = 0
           
                tr_size = fea_list[0].shape[0]
                # ================   training    ============
                while tr_step * batch_size < tr_size:

                    fd1 = {i: d[tr_step * batch_size:(tr_step + 1) * batch_size]
                           for i, d in zip(ftr_in_list, fea_list)}
                    fd2 = {i: d[tr_step * batch_size:(tr_step + 1) * batch_size]
                           for i, d in zip(bias_in_list, biases_list)}
                    fd3 = {lbl_in: y_train[tr_step * batch_size:(tr_step + 1) * batch_size],
                       msk_in: train_mask[tr_step * batch_size:(tr_step + 1) * batch_size],
                       is_train: True,
                       attn_drop: attn_drop_set,
                       ffd_drop: ffd_drop_set}
                    fd = fd1
                    fd.update(fd2)
                    fd.update(fd3)
                    _, loss_value_tr, acc_tr, att_val_train = sess.run([train_op, loss, accuracy, att_val],
                                                                   feed_dict=fd)
                    train_loss_avg += loss_value_tr
                    train_acc_avg += acc_tr
                    tr_step += 1

                vl_step = 0
                vl_size = fea_list[0].shape[0]
                curr_step += 1
                if epoch==nb_epochs-1:
                    saver.save(sess, checkpt_file)

                train_loss_avg = 0
                train_acc_avg = 0
                val_loss_avg = 0
                val_acc_avg = 0
            saver.restore(sess, checkpt_file)
            ts_size = fea_list[0].shape[0]
            ts_step = 0
            ts_loss = 0.0
            ts_acc = 0.0

            while ts_step * batch_size < ts_size:
                fd1 = {i: d[ts_step * batch_size:(ts_step + 1) * batch_size]
                       for i, d in zip(ftr_in_list, fea_list)}
                fd2 = {i: d[ts_step * batch_size:(ts_step + 1) * batch_size]
                       for i, d in zip(bias_in_list, biases_list)}
                fd3 = {lbl_in: y_test[ts_step * batch_size:(ts_step + 1) * batch_size],
                       msk_in: test_mask[ts_step * batch_size:(ts_step + 1) * batch_size],
            
                       is_train: False,
                       attn_drop: attn_drop_set,
                       ffd_drop: ffd_drop_set}
        
                fd = fd1
                fd.update(fd2)
                fd.update(fd3)
                loss_value_ts, acc_ts, jhy_final_embedding = sess.run([loss, accuracy, final_embedding],
                                                                  feed_dict=fd)
                ts_loss += loss_value_ts
                ts_acc += acc_ts
                ts_step += 1

            print('Test loss:', ts_loss / ts_step,
              '; Test accuracy:', ts_acc / ts_step)
            
            #crs.append(ts_loss / ts_step)
            #crs=tuple(crs)
            import pickle
            

            sess.close()
            return(ts_loss / ts_step)

# l2_candi=list([0.02, 0.05, 0.08,0.1,0.2])
l2_candi=list([0.08])
lr_candi=list([0.02])
weight_candi=list([2])

kfolds=2

records=[]
count=0

for p1 in l2_candi:
    for p2 in lr_candi:
        for p3 in weight_candi:
            crs = []
            for fold in range(kfolds):
                imp.reload(gat)
                from gat import GAT, HeteGAT, HeteGAT_multi
                # Create the model with the current set of parameters
                cr = HAN(nb_epochs=100, l2_coef=p1, lr=p2, hid_units=[8], n_heads=[4, 1], ffd_drop_set=0, attn_drop_set=0, fold=fold, weight=p3)
                count += 1
                crs.append(cr)
            
            # Store the parameters and their corresponding results
            records.append({
                'params': {
                    'l2_coef': p1,
                    'lr': p2,
                    'weight': p3
                },
                'results': crs
            })
            print(f'Finished running parameter set: l2_coef={p1}, lr={p2}, weight={p3} @_@')
            
print(f'Total models trained: {count}')            

c:\Users\yunfa\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


KeyboardInterrupt: 

In [ ]:
!pip install tensorflow-gpu==1.15

     -------------------------------------- 294.5/294.5 MB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 3.8/3.8 MB 24.1 MB/s eta 0:00:00
     ------------------------------------- 503.4/503.4 kB 30.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0


In [ ]:
!pip install "tensorflow>=1.15,<2.0"

In [ ]:
#Copy paste print-outs from last block to "xvali.txt" in your "path_result" and save it before running this block!
import os
import numpy as np

path_result='C:\\Users\\yunfa\\180b\\results\\'

os.chdir(path_result)

file1 = open('xvali.txt', 'r')
Lines = file1.readlines()
new_lines=[a.replace('\n','').replace('[','').replace(']','').replace(',','') for a in Lines if a!='\n']
print(len(new_lines))
new_lines1=[]
for a in range(len(new_lines)):
    if new_lines[a][0:4]=='Test':
        new_lines1.append(new_lines[a])
new_lines2=[]
print(len(new_lines1))
for a in range(len(new_lines1)):
    acc=float(new_lines1[a].split('Test accuracy: ')[1])
    new_lines2.append(acc)
print(len(new_lines2))

In [ ]:
#See which parameter combination perform the best
records_agg=[]

#repeats=10
#kfolds=4
for a1 in range(len(l2_candi)):
    r1=[]
    for a2 in range(len(lr_candi)):
        r2=[]
        for a3 in range(len(weight_candi)):
            r3=[]
            for a4 in range(repeats):
                r4=[]
                for a5 in range(kfolds): 
                    r4.append(new_lines2[0])
                    new_lines2.pop(0)
                r3.append(r4)
            r2.append(r3)
        r1.append(r2)
    records_agg.append(r1)


r=np.array(records_agg,dtype='float')
print(r.shape)
#print(r)
w=np.mean(r,axis=3)
print(w.shape)
e=np.mean(w,axis=3)
print(e.shape)
pos=np.where(e== e.max())
print(e)
print(pos)

print('l2:'+str(l2_candi[pos[0][0]]))
print('lr:'+str(lr_candi[pos[1][0]]))
print('weight:'+str(weight_candi[pos[2][0]]))

In [ ]:
print(weight_candi)
print((lr_candi))
print(l2_candi)

print(repeats)
print(kfolds)